## 파이썬 완정정복 반보형과 반복자


In [1]:
import sys

In [2]:
sys.version_info

sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)

### 반복자 언패킹 처리 

In [3]:
l = [1,2,3,4]

In [4]:
m = map(lambda x : x**2, l)

In [5]:
print(type(m))

<class 'map'>


In [6]:
f = filter(lambda x : x> 3, l)

In [7]:
print(type(f))

<class 'filter'>


#### 상속관계를 확인하면 반복형이면서 반복자이다

In [8]:
import collections.abc as abc

In [9]:
issubclass(type(m), abc.Iterator)

True

In [10]:
issubclass(type(m), abc.Iterable)

True

In [11]:
issubclass(type(f), abc.Iterator)

True

In [12]:
issubclass(type(f), abc.Iterable)

True

### map/filter 리스트 리터럴 내부에서  실행 기준

#### *표를 붙이면 반복자가 실행된다

In [13]:
[*map(lambda x : x**2, l)]

[1, 4, 9, 16]

In [14]:
[*filter(lambda x : x>2, l)]

[3, 4]

#### 실제 별표를 붙이지 않으면 객체만 만들어진다

In [15]:
[map(lambda x : x**2, l)]

In [16]:
[filter(lambda x : x>2, l)]

### 반복형 추상 클래스 확인 

In [17]:
import collections.abc as cols
import pprint

pprint.pprint(cols.Iterable.__dict__)

mappingproxy({'__abstractmethods__': frozenset({'__iter__'}),
              '__doc__': None,
              '__iter__': <function Iterable.__iter__ at 0x0000000002811F28>,
              '__module__': 'collections.abc',
              '__slots__': (),
              '__subclasshook__': <classmethod object at 0x000000000249FB70>,
              '_abc_cache': <_weakrefset.WeakSet object at 0x000000000249FBE0>,
              '_abc_negative_cache': <_weakrefset.WeakSet object at 0x00000000063833C8>,
              '_abc_negative_cache_version': 51,
              '_abc_registry': <_weakrefset.WeakSet object at 0x000000000249FBA8>})


### 반복자 추상클래스 확인 

In [18]:
import collections.abc as cols
import pprint

pprint.pprint(cols.Iterator.__dict__)

mappingproxy({'__abstractmethods__': frozenset({'__next__'}),
              '__doc__': None,
              '__iter__': <function Iterator.__iter__ at 0x00000000028162F0>,
              '__module__': 'collections.abc',
              '__next__': <function Iterator.__next__ at 0x0000000002816268>,
              '__slots__': (),
              '__subclasshook__': <classmethod object at 0x000000000249FD30>,
              '_abc_cache': <_weakrefset.WeakSet object at 0x000000000249FDA0>,
              '_abc_negative_cache': <_weakrefset.WeakSet object at 0x0000000006383390>,
              '_abc_negative_cache_version': 51,
              '_abc_registry': <_weakrefset.WeakSet object at 0x000000000249FD68>})


### 반복형과 반복자 메소드 차이 확인 

In [19]:
import collections.abc as cols

able = set(dir(cols.Iterable))
ator = set(dir(cols.Iterator))

print(ator - able)

{'__next__'}


#### 문자열이 반복형과 반복자인지 구분

In [20]:
import collections.abc as cols

s = "Iterator"

print(isinstance(s, cols.Iterable))
print(isinstance(s, cols.Iterator))

True
False


In [21]:
print(str.__dict__['__iter__'])


<slot wrapper '__iter__' of 'str' objects>


In [22]:
try :
    print(str.__dict__['__next__'])
except Exception as e :
    print(e)

'__next__'


### 내장 자료형을 반복자로 처리해보기

#### 문자열을 반복자로 변환하기 

In [23]:
import collections.abc as cols

s = "Hello"

siter = iter(s)

In [24]:
siter

In [25]:
print(next(siter))
print(next(siter))
print(next(siter))
print(next(siter))
print(next(siter))

print(next(siter, None))

H
e
l
l
o
None


#### 리스트를 반복자로 전환하기

In [26]:
l = [1, 2, 3, 4]

lator = iter(l)

print(lator)
print(type(lator))

<class 'list_iterator'>


#### 튜플을 반복자로 처리하기

In [27]:
t = (1, 2, 3, 4)

tator = iter(t)

print(tator)
print(type(tator))

<class 'tuple_iterator'>


#### 딕셔너리를 반복자로 변환 

In [28]:
d = {'a': 1}
dator = iter(d)

print(dator)
print(type(dator))

<class 'dict_keyiterator'>


#### 집합을 반복자로 변환

In [29]:
s = {1, 2, 3}

sator = iter(s)

print(sator)
print(type(sator))

<class 'set_iterator'>


### 사용자 정의 반복자 만들기 

In [30]:
import collections.abc as cabc
import abc


class Iterator(cabc.Iterable):
    __slots__ = ()
    
    @abc.abstractmethod
    def __next__(self):
        "Return the next item from the iterator. When exhausted, raise StopIteration"
        raise StopIteration

    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" is B.__dict__ for B in C.__mro__) and
               any("__iter__" is B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented

#### 리스트를 이용해서 반복자 확인하기 

In [31]:
import collections.abc as abc


x =  [1, 2, 3, 4]
print(issubclass(type(x), abc.Iterable))
print(issubclass(type(x), abc.Iterator))
print(type(x))

xi = iter(x)
print(type(xi))

True
False
<class 'list'>
<class 'list_iterator'>


####  리스트에는 반복자 메소드가 존재하지 않는다

In [32]:
try :
    list.__dict__['iter']
except Exception as e :
    print(e)

'iter'


#### 사용자 반복자를 이용할 경우 상속관계가 없음

In [33]:
print(issubclass(type(x), Iterator))

False


In [34]:
print(issubclass(type(iter(x)), Iterator))

False


#### 사용자 정의한 클래스를 만들고 처리 


In [35]:
class ITOR(Iterator) :
    def __iter__(self) :
        pass
    def __next__(self) :
        pass

In [36]:
print(issubclass(ITOR, Iterator))

True


### 반복형 확인  : iter

In [37]:
from collections import abc

class Foo:
    def __iter__(self):
        pass
    
    
print(issubclass(Foo, abc.Iterable))    
f = Foo()
print(isinstance(f, abc.Iterable))

True
True


### 반복형 확인 : getitem

In [38]:
from collections import abc

class Foo:
    def __getitem__(self, index):
        pass
    
    
print(issubclass(Foo, abc.Iterable))    
f = Foo()
print(isinstance(f, abc.Iterable))

sf = iter(f)

print(isinstance(sf, abc.Iterable))

False
False
True


### 반복형 함수 알아보기

In [39]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



In [40]:
help(next)

Help on built-in function next in module builtins:

next(...)
    next(iterator[, default])
    
    Return the next item from the iterator. If default is given and the iterator
    is exhausted, it is returned instead of raising StopIteration.



In [41]:
x = 0
xc = lambda : x+1
xx = iter(xc, 3)
print(next(xx))
x += 1
print(next(xx))
x += 1
print(next(xx, None))
x += 1

1
2
None


### 반복형 문장 클래스를 만들기 

In [42]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence():
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return "Sentence(%s)" %reprlib.repr(self.text)
    

In [43]:
print(dir(Sentence))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


In [44]:

s3 = Sentence("Pig and Pepper")
print(s3.words)

['Pig', 'and', 'Pepper']


#### 반복자로 처리 하기

In [45]:
s4 = Sentence("Pig and Pepper")
it = iter(s4)
print(dir(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it,None))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']
Pig
and
Pepper
None


#### 반복자 만들기 

In [46]:
import collections.abc as cols


class SeqIterator:
    def __init__(self, seq):
        self.seq = seq
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n < len(self.seq):
            result = self.seq[self.n]
            self.n += 1
            return result
        else:
            raise StopIteration

#### 반복자 생성해서 처리하기

In [47]:
s = SeqIterator("반복자처리")
siter = iter(s)

print(siter)
print(issubclass(type(siter), cols.Iterator))

True


In [48]:
for i in siter:
    print(i)

반
복
자
처
리


#### 관계 확인하기

In [49]:
s = SeqIterator("반복자처리")
siter = iter(s)

print(siter)
print(issubclass(type(siter), cols.Iterator))


True


In [50]:
print(next(siter, None))
print(next(siter, None))
print(next(siter, None))
print(next(siter, None))
print(next(siter, None))
print(next(siter, None))

반
복
자
처
리
None


### 반복자에 대한 모듈 알아보기

In [51]:
import itertools as it


for i in dir(it):
    if not i.startswith("_"):
        print(i)

accumulate
chain
combinations
combinations_with_replacement
compress
count
cycle
dropwhile
filterfalse
groupby
islice
permutations
product
repeat
starmap
takewhile
tee
zip_longest


#### 반복자 관계 확인하기 

In [52]:
import itertools as it
import collections.abc as cols
import operator as op


print(it.accumulate)
print(issubclass(it.accumulate, cols.Iterator))

a = it.accumulate([x for x in range(5)])
print(list(a))

<class 'itertools.accumulate'>
True
[0, 1, 3, 6, 10]


#### 체인 함수 사용하기

In [53]:
import itertools

gen = itertools.chain('ABC', 'DEF')
for i in gen:
    print(i, end=" ")
    
print()

gen1 = itertools.chain.from_iterable(['ABC', 'DEf'])
for i in gen1:
    print(i, end=" ")

A B C D E F 
A B C D E f 

#### 퍼뮤테이션 함수 생성하기 

In [54]:
import itertools

gen = itertools.permutations([1, 2, 3], 2)
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen,None))

(1, 2)
(1, 3)
(2, 1)
(2, 3)
(3, 1)
(3, 2)
None


#### 순서쌍을 만들기 

In [55]:
import itertools

gen = itertools.product([1, 2, 3], [1, 2, 3])

for i in gen:
    print(i)

(1, 1)
(1, 2)
(1, 3)
(2, 1)
(2, 2)
(2, 3)
(3, 1)
(3, 2)
(3, 3)


#### 조합을 만들기

In [56]:
import itertools

gen = itertools.combinations([1, 2, 3], 2)
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen, None))

(1, 2)
(1, 3)
(2, 3)
None
